In [109]:
import pandas as pd
import numpy as np
from Abbott_lookup import ab_lookup 
from product_info_JSON import PRODUCT_INFORMATION

# Create a Lookup Table to manually replace missing values in Abbot Product Table
# This can be mannually updated with research projects
construct = ab_lookup  #Just to get this out of the analysis page
# Create a dataframe from constructed table above
lookup = pd.DataFrame(data=construct)
# Create an array from lookup table to help filter later on
lookup_filter = lookup["item_type"].array
lookup.drop(["name"], axis=1, inplace=True)
lookup = lookup.set_index("item_type")

# ---------------------------- End of Lookup Table work ---------------------------------

# Create the Abbott product table

Abbot_products_original = pd.read_csv("../Data/Abbott/Abbott_scrape_data.csv",  usecols=PRODUCT_INFORMATION.keys()) #This could be replaced with output from other parser
Abbot_products = Abbot_products_original.copy() #to be used by future analysis

len(Abbot_products)

315

In [110]:
# just used to explore the unique values in the columns so I could make the XML file with all the look up values

import math
import re
from collections.abc import Iterable
vals = Abbot_products['size_or_weight'].str.split(' ') #extract(r'([a-zA-Z]+)')
s = ''
for i in vals:
    if isinstance(i, Iterable):
        s += ' '.join(i)
s = s.split(' ')
s = set(s)


In [111]:
#import each look up list - will be used to clean the data in next cell

import xml.etree.ElementTree as ET
#get all types and all measurements
root = ET.parse('../Data/Abbott/Abbott_product_types_lookup.xml').getroot()
types_list = [i.text for i in root[0]]
types_list
measure_list = [i.text for i in root[1]]
measure_list
bulk_form_list = [i.text for i in root[2]]


In [112]:
#extract value from list that is found in another list
def exists_in_list(l, val):
    for i in val:
        if i in l:
            return i
    return None


a = Abbot_products['size_or_weight']
split_data = list()
#splits by '/'
for i in a:
    new_tuple = str(i).rstrip().split('/')
    split_data.append(new_tuple)
    
    
lhs = list()
rhs = list()

#lhs to have weight metrics, type and weight number
#Rhs will have bulk sales availablility
for i in split_data:
    if len(i) == 2:
        rhs.append(i[1])
        lhs.append(i[0])
    else:
        lhs.append(i[0])
        rhs.append('')

wt_metric = list()
pr_type = list()
wt = list()
bulk_sales_type = list()
bulk_sales_nums = list()

#will create all the same length lists but with their specific values
for i in lhs:
    v = exists_in_list(i, measure_list)
    wt_metric.append(v)
    v = exists_in_list(i, types_list)
    pr_type.append(v)
    d = i.split(' ')
    prod_wt_num = [el for el in d if bool(re.search(r'\d', el))]
    
    if prod_wt_num:
        wt.append(prod_wt_num[0])
    else:
        wt.append('')


for i in rhs:
    v = exists_in_list(i, bulk_form_list)
    bulk_sales_type.append(v)
    d = i.split(' ')
    prod_wt_num = [el for el in d if bool(re.search(r'\d', el))]
    
    if prod_wt_num:
        bulk_sales_nums.append(prod_wt_num[0])
    else:
        bulk_sales_nums.append('')

    


In [113]:
Abbot_products.head()

,url,store,name,price,size_or_weight,availability,item_type,description,ingredients,allergin_info,...,Form,Flavours,Cans Type X,Cans Type Y,clinical indications,benefits,clinical_indications,feature_table_rows,usage,entry_date
0,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,30.8 oz can,In stock,SKU#:66439e,"Due to high demand, to ensure availability for...","Nonfat Milk, Lactose, High Oleic Safflower Oil...",Contains milk and soy ingredients. ...,...,Powder,NaN,SELECT 4 CANS,SELECT 2 CANS,NaN,NaN,NaN,NaN,NaN,21/04/2020 16:52:52
1,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,30.8 oz can,In stock,SKU#:66439e,"Due to high demand, to ensure availability for...","Nonfat Milk, Lactose, High Oleic Safflower Oil...",Contains milk and soy ingredients. ...,...,Powder,NaN,SELECT 4 CANS,SELECT 2 CANS,NaN,NaN,NaN,NaN,NaN,21/04/2020 17:03:37
2,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Ready-to-Feed,$57.96,2 fl oz bottle / Case of 48,In stock,SKU#:67436,"Due to high demand, to ensure availability for...","Water, Nonfat Milk, Lactose, High Oleic Safflo...",Contains milk and soy ingredients. ...,...,Ready-to-Feed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21/04/2020 17:03:37
3,https://abbottstore.com/advancepowder-30-8ozca...,Abbott,Similac Advance Infant Formula Powder,$36.49,30.8 oz can,In stock,SKU#:58585e,Similac Advance is designed to support your ba...,"Nonfat Milk, Lactose, Whey Protein Concentrate...",Contains milk and soy ingredients. ...,...,Powder,NaN,SELECT 4 CANS,SELECT 2 CANS,NaN,NaN,NaN,NaN,NaN,21/04/2020 17:03:37
4,https://abbottstore.com/similac-pro-sensitive-...,Abbott,Similac Pro-Sensitive Infant Formula Powder,$165.99,29.8 oz can / Case of 4,BACKORDER,SKU#:66441,Similac Pro-Sensitive is the first infant form...,"Corn Syrup, Milk Protein Isolate, High Oleic S...",Contains milk ingredients. ...,...,Powder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21/04/2020 17:03:37


In [114]:

can_numbers_1 = Abbot_products['Cans Type X'].str.extract('(\d+)')
can_numbers_2 = Abbot_products['Cans Type Y'].str.extract('(\d+)')
can_numbers = can_numbers_1 + ', ' + can_numbers_2
can_numbers = can_numbers.values.tolist()
#Abbot_products['product_type_counts'] = can_numbers
for i in range(0, len(can_numbers)):
    #print(can_numbers[i][0])
    if isinstance(can_numbers[i][0], str):
        bulk_sales_nums[i] = can_numbers[i][0]
        bulk_sales_type[i] = 'can'
        #print(can_numbers[i])


In [115]:
Abbot_products['product_format'] = pr_type
Abbot_products['product_measurement_metric'] = wt_metric
Abbot_products['product_measurement_numeric'] = wt
Abbot_products['bulk_sales_type'] = bulk_sales_type
Abbot_products['bulk_sales_numeric'] = bulk_sales_nums


In [116]:

Abbot_products.head()

,url,store,name,price,size_or_weight,availability,item_type,description,ingredients,allergin_info,...,benefits,clinical_indications,feature_table_rows,usage,entry_date,product_format,product_measurement_metric,product_measurement_numeric,bulk_sales_type,bulk_sales_numeric
0,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,30.8 oz can,In stock,SKU#:66439e,"Due to high demand, to ensure availability for...","Nonfat Milk, Lactose, High Oleic Safflower Oil...",Contains milk and soy ingredients. ...,...,NaN,NaN,NaN,NaN,21/04/2020 16:52:52,can,oz,30.8,can,"4, 2"
1,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,30.8 oz can,In stock,SKU#:66439e,"Due to high demand, to ensure availability for...","Nonfat Milk, Lactose, High Oleic Safflower Oil...",Contains milk and soy ingredients. ...,...,NaN,NaN,NaN,NaN,21/04/2020 17:03:37,can,oz,30.8,can,"4, 2"
2,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Ready-to-Feed,$57.96,2 fl oz bottle / Case of 48,In stock,SKU#:67436,"Due to high demand, to ensure availability for...","Water, Nonfat Milk, Lactose, High Oleic Safflo...",Contains milk and soy ingredients. ...,...,NaN,NaN,NaN,NaN,21/04/2020 17:03:37,bottle,fl oz,2,Case,48
3,https://abbottstore.com/advancepowder-30-8ozca...,Abbott,Similac Advance Infant Formula Powder,$36.49,30.8 oz can,In stock,SKU#:58585e,Similac Advance is designed to support your ba...,"Nonfat Milk, Lactose, Whey Protein Concentrate...",Contains milk and soy ingredients. ...,...,NaN,NaN,NaN,NaN,21/04/2020 17:03:37,can,oz,30.8,can,"4, 2"
4,https://abbottstore.com/similac-pro-sensitive-...,Abbott,Similac Pro-Sensitive Infant Formula Powder,$165.99,29.8 oz can / Case of 4,BACKORDER,SKU#:66441,Similac Pro-Sensitive is the first infant form...,"Corn Syrup, Milk Protein Isolate, High Oleic S...",Contains milk ingredients. ...,...,NaN,NaN,NaN,NaN,21/04/2020 17:03:37,can,oz,29.8,Case,4


In [117]:
# ------------- Setup dataframe and columns -----------------

Abbot_products['id'] = Abbot_products['item_type']

del Abbot_products['item_type']
del Abbot_products['benefits']
del Abbot_products['clinical_indications']
del Abbot_products['feature_table_rows']
del Abbot_products['usage']
del Abbot_products['Cans Type X']
del Abbot_products['Cans Type Y'] 
del Abbot_products['Flavours']
del Abbot_products['size_or_weight']
del Abbot_products['availability'] 
del Abbot_products['Sizes']
del Abbot_products['footnotes']
del Abbot_products['ingredients']

In [118]:
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.split('(', expand=True)[0]
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.strip()
df = Abbot_products['serving_size_1'].str.split(' ', expand=True)

Abbot_products['serving_size_metric'] = df[1]
Abbot_products['serving_size_numeric'] = df[0]

for i in range(1, 6):
    del Abbot_products['serving_size_' + str(i)]

for i in range(1, 6):
    del Abbot_products['nutrient_table_' + str(i)]

Abbot_products.head()

,url,store,name,price,description,allergin_info,vitamin_table_1,vitamin_table_2,vitamin_table_3,vitamin_table_4,...,clinical indications,entry_date,product_format,product_measurement_metric,product_measurement_numeric,bulk_sales_type,bulk_sales_numeric,id,serving_size_metric,serving_size_numeric
0,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,"Due to high demand, to ensure availability for...",Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '2027', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 16:52:52,can,oz,30.8,can,"4, 2",SKU#:66439e,Cal,100
1,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,"Due to high demand, to ensure availability for...",Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '2027', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,can,oz,30.8,can,"4, 2",SKU#:66439e,Cal,100
2,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Ready-to-Feed,$57.96,"Due to high demand, to ensure availability for...",Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '2027', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,bottle,fl oz,2,Case,48,SKU#:67436,Cal,100
3,https://abbottstore.com/advancepowder-30-8ozca...,Abbott,Similac Advance Infant Formula Powder,$36.49,Similac Advance is designed to support your ba...,Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '1929', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,can,oz,30.8,can,"4, 2",SKU#:58585e,Cal,100
4,https://abbottstore.com/similac-pro-sensitive-...,Abbott,Similac Pro-Sensitive Infant Formula Powder,$165.99,Similac Pro-Sensitive is the first infant form...,Contains milk ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '1929', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,can,oz,29.8,Case,4,SKU#:66441,Cal,100


In [119]:



Abbot_products.drop_duplicates(keep='first', inplace=True)
#Abbot_products[pd.isnull(Abbot_products['product_format'])]
Abbot_products.head()

,url,store,name,price,description,allergin_info,vitamin_table_1,vitamin_table_2,vitamin_table_3,vitamin_table_4,...,clinical indications,entry_date,product_format,product_measurement_metric,product_measurement_numeric,bulk_sales_type,bulk_sales_numeric,id,serving_size_metric,serving_size_numeric
0,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,"Due to high demand, to ensure availability for...",Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '2027', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 16:52:52,can,oz,30.8,can,"4, 2",SKU#:66439e,Cal,100
1,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Powder,$41.49,"Due to high demand, to ensure availability for...",Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '2027', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,can,oz,30.8,can,"4, 2",SKU#:66439e,Cal,100
2,https://abbottstore.com/similac-pro-advance-in...,Abbott,Similac Pro-Advance Infant Formula Ready-to-Feed,$57.96,"Due to high demand, to ensure availability for...",Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '2027', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,bottle,fl oz,2,Case,48,SKU#:67436,Cal,100
3,https://abbottstore.com/advancepowder-30-8ozca...,Abbott,Similac Advance Infant Formula Powder,$36.49,Similac Advance is designed to support your ba...,Contains milk and soy ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '1929', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,can,oz,30.8,can,"4, 2",SKU#:58585e,Cal,100
4,https://abbottstore.com/similac-pro-sensitive-...,Abbott,Similac Pro-Sensitive Infant Formula Powder,$165.99,Similac Pro-Sensitive is the first infant form...,Contains milk ingredients. ...,"{'Vitamin A, IU': '300', 'Vitamin A, IU(DV)': ...","{'Vitamin A, IU': '1929', 'Vitamin A, IU(DV)':...",NaN,NaN,...,NaN,21/04/2020 17:03:37,can,oz,29.8,Case,4,SKU#:66441,Cal,100


In [120]:
Abbot_products['Form'][0]

'Powder'

In [121]:

# Send to CSV
Abbot_products.to_csv("../Data/Abbott/Abbot_products.csv")


In [122]:
# -------------------------------------------

# Set up table which shows all the ingredients for products vertically for analytics
Abbot_products_ingredients_item_type = Abbot_products_original

# Changed from Name to SKU ID "item_type"
Abbot_products_ingredients_item_type = Abbot_products_ingredients_item_type[pd.notnull(Abbot_products_ingredients_item_type['item_type'])]
Abbot_products_ingredients = Abbot_products_ingredients_item_type[['item_type','ingredients']]
Abbot_products_name = Abbot_products_ingredients_item_type['item_type']
Abbot_main_ingredients = pd.concat([Abbot_products_name,
                                    Abbot_products_ingredients['ingredients'].str.split(', ', expand=True)], axis=1)
Abbot_main_ingredients = pd.melt(Abbot_main_ingredients, id_vars = ["item_type"])
Abbot_main_ingredients.dropna(inplace=True)
Abbot_main_ingredients = Abbot_main_ingredients[(Abbot_main_ingredients['value'] != 0)]

Abbot_main_ingredients.rename(columns={'value' : 'ingredient'},inplace=True)
del Abbot_main_ingredients['variable']
Abbot_main_ingredients.head()
Abbot_main_ingredients.sort_values('item_type', inplace=True, ascending=True)
Abbot_main_ingredients.drop_duplicates(keep='first', inplace=True)
Abbot_main_ingredients


# Remove white space and "AND" 
Abbot_main_ingredients.ingredient = Abbot_main_ingredients.ingredient.str.strip()
Abbot_main_ingredients['ingredient'] = Abbot_main_ingredients['ingredient'].str.replace("and ", "", regex=False)
Abbot_main_ingredients['ingredient'] = Abbot_main_ingredients['ingredient'].str.replace(".", "", regex=False)

# Send to CSV
Abbot_main_ingredients.to_csv("../Data/Abbott/Abott_products_ingredients.csv")

In [123]:

# -----------

# Set up table which shows all the flavours for products vertically for analytics
# Changed name to SKU "item_type"

Abbot_product_flavours_item_type = Abbot_products_original
 #This could be replaced with output from other parser
    
Abbot_product_flavours_item_type = Abbot_product_flavours_item_type[pd.notnull(Abbot_product_flavours_item_type['item_type'])]
Abbot_products_name = Abbot_product_flavours_item_type['item_type']
Abbot_product_flavours = Abbot_product_flavours_item_type[['item_type','Flavours']]
Abbot_product_flavours = pd.concat([Abbot_products_name,
                                    Abbot_product_flavours['Flavours'].str.split(', ', expand=True)], axis=1)


Abbot_product_flavours = pd.melt(Abbot_product_flavours, id_vars = ["item_type"])
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].astype(str)

Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("]", "", regex=False)
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("[", "", regex = False)
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("'", "", regex=False)
Abbot_product_flavours.dropna(inplace=True)
Abbot_product_flavours = Abbot_product_flavours[(Abbot_product_flavours['value'] != 0)]
del Abbot_product_flavours['variable']
Abbot_product_flavours.rename(columns={'value' : 'Flavours'},inplace=True)

Abbot_product_flavours.sort_values('item_type', inplace=True, ascending=True)
Abbot_product_flavours.drop_duplicates(keep='first', inplace=True)
Abbot_product_flavours

Abbot_product_flavours.to_csv("../Data/Abbott/Abott_products_flavours.csv")
